# Práctica 3: Representación del conocimiento

__Fecha de entrega: 15 de mayo de 2022__

El objetivo de esta práctica es aplicar los conceptos teóricos vistos en clase en el módulo de Representación de conocimiento. La práctica consta de un único notebook que se entregará en la tarea de entrega habilitada en el Campus  Virtual.

Cada consulta debe contener breves comentarios que expliquen cada tripleta. __No se valorarán consultas sin explicaciones__.

__Número de grupo: 02__

__Nombres de los estudiantes: Jesus Martin y Jorge Arevalo__


## Consultas SPARQL sobre Wikidata.

En esta práctica vamos a usar el [punto de acceso SPARQL](https://query.wikidata.org/) de Wikidata para contestar las preguntas que se formulan a continuación. Cada pregunta debe ser respondida realizando una única consulta SPARQL. Para cada una de las entidades recuperadas se mostrará __tanto su identificador como su etiqueta__ (nombre de la entidad en lenguaje natural). 

Para cada una de las preguntas debes mostrar tanto la consulta como la respuesta obtenida. Si lo consideras necesario, puedes añadir celdas adicionales en formato _Markdown_ para explicar decisiones que hayas tomado al crear la consulta o cualquier otro dato que consideres interesante.
 
Para resolver estas consultas puedes usar __dos recursos que te recomendamos consultar__ son:

- [Este tutorial de SPARQL](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial).
- [Esta recopilación de ejemplos](https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples)

In [1]:
## Lo primero sería instalar en tu entorno de Python wdsparql
## Si no lo tienes instalado, puedes hacerlo desde el notebook
import sys

!{sys.executable} -m pip install wdsparql

In [2]:
# Asumismos instalado wdsparql
%load_ext autoreload
%load_ext wdsparql
%autoreload 2

### Ejemplo

Recuperar todas las instancias directas de la clase [Cabra (Q2934)](https://www.wikidata.org/wiki/Q2934) que aparecen en la base de conocimiento.

In [3]:
%%wdsparql
SELECT ?item ?itemLabel 
WHERE 
{
  ?item wdt:P31 wd:Q2934.  # instancias directas de la clase Cabra
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,item,itemLabel
0,http://www.wikidata.org/entity/Q151345,Billygoat Hennes
1,http://www.wikidata.org/entity/Q3569037,William Windsor
2,http://www.wikidata.org/entity/Q23003932,His Whiskers
3,http://www.wikidata.org/entity/Q24287064,Taffy
4,http://www.wikidata.org/entity/Q41239734,Lance Corporal Shenkin III
5,http://www.wikidata.org/entity/Q41240892,Lance Corporal Shenkin II
6,http://www.wikidata.org/entity/Q41241416,Lance Corporal Shenkin I


### Consulta 1

[Isaac Asimov (Q34981)](https://www.wikidata.org/wiki/Q34981) fue un escritor y profesor de bioquímica en la Universidad de Boston conocido por ser un prolífico autor de obras de ciencia ficción, historia y divulgación científica. 

Vamos a comenzar por averiguar su fecha y lugar de nacimiento (localidad y país al que pertenece la localidad en la actualidad).

In [4]:
%%wdsparql
SELECT ?fecha ?ciudadLabel ?paisLabel
WHERE
{
  #Isaac Asimov      #fecha de nacimiento
  wd:Q34981 wdt:P569 ?fecha;
                     #ciudad de nacimiento
            wdt:P19  ?ciudad.
                     #pais al cual pertenece la ciudad de nacimiento
  ?ciudad   wdt:P17  ?pais.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}

,fecha,ciudadLabel,paisLabel
0,1920-01-02T00:00:00Z,Petrovichi,Russia


### Consulta 2

A continuación vamos a averiguar todas las distintas profesiones (ocupaciones) que se le reconocen en la base de conocimiento. Queremos obtener los resultados ordenados alfabéticamente por el nombre de la profesión.

In [5]:
%%wdsparql
SELECT ?profesionLabel
WHERE
{
   #Isaac Asimov     #profesion
  wd:Q34981 wdt:P106 ?profesion.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
ORDER BY ASC(?profesionLabel)

,profesionLabel
0,autobiographer
1,biochemist
2,journalist
3,non-fiction writer
4,novelist
5,prosaist
6,science fiction writer
7,science writer
8,scientist
9,screenwriter


### Consulta 3

De todas esas profesiones, ¿cuáles corresponden con subtipos de [Escritor (Q36180)](https://www.wikidata.org/wiki/Q36180)? Ten en cuenta que la jerarquía de tipos de escritores puede ser muy sofisticada.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Property_paths) te resulte útil.

In [6]:
%%wdsparql
SELECT ?profesionLabel
WHERE
{
  #Hayamos todas las profesiones de Asimov
  wd:Q34981 wdt:P106 ?profesion.
  #Nos quedamos solo con las que sean subtipo(wdt:P279) de Escritores(wd:Q36180)
  ?profesion wdt:P279 wd:Q36180.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
ORDER BY ASC(?profesionLabel)

,profesionLabel
0,journalist
1,non-fiction writer
2,novelist
3,prosaist
4,science writer
5,screenwriter


### Consulta 4

Asimov se casó más de una vez. Para cada uno de los matrimonios queremos conocer el nombre de la esposa y las fechas de inicio y finalización. Los resultados deben aparecer ordenados cronológicamente.

Para resolver esta consulta necesitarás acceder a los cualificadores de nodos sentencia y necesitarás entender los prefijos que usa Wikidata. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Qualifiers) te resulte útil.

In [7]:
%%wdsparql
SELECT ?esposa ?esposaLabel ?fechaInicio ?fechaFinal
WHERE
{
  #Para cada conyuge(p:P26) accedemos a las fechas de inicio y fin y nombre de la esposa
  #con esta sintaxis accedemos al nodo de la declaracion de un matrimonio, y sacamos los datos que nos interesen de ese matrimonio
  wd:Q34981 p:P26 [pq:P580 ?fechaInicio; pq:P582 ?fechaFinal; ps:P26 ?esposa]
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
ORDER BY ASC(?fechaInicio)

,esposa,esposaLabel,fechaInicio,fechaFinal
0,http://www.wikidata.org/entity/Q107693884,Gertrude Asimov,1942-07-26T00:00:00Z,1973-01-01T00:00:00Z
1,http://www.wikidata.org/entity/Q517435,Janet Asimov,1973-01-01T00:00:00Z,1992-01-01T00:00:00Z


### Consulta 5

Asimov recibió muchos premios a lo largo de su carrera. Queremos obtener la lista de premios y, para cada uno de ellos, la fecha y la obra por la que fue premiado (si están disponibles). Los resultados se deben mostrar ordenador por fecha.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#OPTIONAL) te resulte útil.

In [8]:
%%wdsparql
SELECT ?premio ?tituloLabel ?fecha ?obraLabel
WHERE
{
  #Premios que obtuvo Asimov
  wd:Q34981 p:P166 ?premio.
  #Titulo del premio
  ?premio ps:P166 ?titulo.
  #Obtenemos la fecha en la que obtuvo el premio y la obra si estan disponibles, si no aparecera vacio
  OPTIONAL { ?premio pq:P585 ?fecha. }
  OPTIONAL { ?premio pq:P1686 ?obra. }
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
ORDER BY ASC(?fecha)

,premio,tituloLabel,fecha,obraLabel
0,http://www.wikidata.org/entity/statement/Q3498...,Fellow of the Committee for Skeptical Inquiry,None,None
1,http://www.wikidata.org/entity/statement/Q3498...,AAAS Fellow,None,None
2,http://www.wikidata.org/entity/statement/Q3498...,Hugo Award,1963-01-01T00:00:00Z,None
3,http://www.wikidata.org/entity/statement/Q3498...,James T. Grady-James H. Stack Award for Interp...,1965-01-01T00:00:00Z,None
4,http://www.wikidata.org/entity/statement/Q3498...,Hugo Award,1966-01-01T00:00:00Z,Foundation Trilogy
5,http://www.wikidata.org/entity/statement/Q3498...,Edward E. Smith Memorial Award,1967-01-01T00:00:00Z,None
6,http://www.wikidata.org/entity/statement/Q3498...,Nebula Award for Best Novel,1972-01-01T00:00:00Z,The Gods Themselves
7,http://www.wikidata.org/entity/statement/Q3498...,Hugo Award for Best Novel,1973-01-01T00:00:00Z,The Gods Themselves
8,http://www.wikidata.org/entity/statement/Q3498...,Locus Award for Best Novel,1973-01-01T00:00:00Z,The Gods Themselves
9,http://www.wikidata.org/entity/statement/Q3498...,Klumpke-Roberts Award,1975-01-01T00:00:00Z,None


### Consulta 6

Ahora queremos conocer todas las obras escritas por Asimov del tipo [obras literarias de ciencia ficción (Q3238422)](https://www.wikidata.org/wiki/Q3238422) y cualquiera de sus subgéneros. Los resultados se deben mostrar ordenados por nombre.

No pasa nada si en el resultado aparecen mezclados novelas, relatos, colecciones...

In [9]:
%%wdsparql
SELECT ?obra ?obraLabel
WHERE
{
  #Obtenemos las obras de ciencia ficcion
  ?obra wdt:P136 wd:Q24925;
  #Que hayan sido escritas por isaac asimov
        wdt:P50 wd:Q34981.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
ORDER BY ASC(?obraLabel)

,obra,obraLabel
0,http://www.wikidata.org/entity/Q104868766,Q104868766
1,http://www.wikidata.org/entity/Q104868864,Q104868864
2,http://www.wikidata.org/entity/Q104868972,Q104868972
3,http://www.wikidata.org/entity/Q104869051,Q104869051
4,http://www.wikidata.org/entity/Q16069681,Q16069681
5,http://www.wikidata.org/entity/Q25492175,Q25492175
6,http://www.wikidata.org/entity/Q42723389,Q42723389
7,http://www.wikidata.org/entity/Q42723496,Q42723496
8,http://www.wikidata.org/entity/Q54826108,Q54826108
9,http://www.wikidata.org/entity/Q98141420,Q98141420


### Consulta 7

Vamos a volver a recuperar las mismas obras de la consulta anterior pero mostrando también la fecha de publicación de cada una. Las obras deben aparece aunque no tengan fecha de publicación asociada. En caso de que una obra tenga varias fechas de publicación, se mostrará sólo la más antigua. Los resultados deben aparecer en orden cronológico.

Para calcular la fecha correcta tendrás que agrupar las respuestas por obra y aplicar una función de agregación sobre las fechas de publicación. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Grouping) te resulte útil.

Las obras como https://www.wikidata.org/wiki/Q21190269 que tiene dos fechas de publicacion ahora nos muestra la mas antigua

In [10]:
%%wdsparql
#Selecionamos la fecha de publicacion mas antigua de cada obra
SELECT ?obra ?obraLabel (MIN(?fecha) AS ?minFecha)
WHERE
{
  #Publicaciones de ciencia ficcion de Asimov
  ?obra wdt:P136 wd:Q24925;
        wdt:P50 wd:Q34981.
  #Mostramos la fecha de publicacion si esta disponible
  OPTIONAL { ?obra wdt:P577 ?fecha. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
#Agrupamos por obra para seleccionar la fecha de publicacion mas antigua
GROUP BY ?obra ?obraLabel
ORDER BY ?minFecha

,obra,obraLabel,minFecha
0,http://www.wikidata.org/entity/Q368390,Galactic Empire,None
1,http://www.wikidata.org/entity/Q913482,Robot series,None
2,http://www.wikidata.org/entity/Q3222595,Key Item,None
3,http://www.wikidata.org/entity/Q110270091,The Encyclopedists,None
4,http://www.wikidata.org/entity/Q7742168,The Instability,None
5,http://www.wikidata.org/entity/Q15917684,The Early Asimov,None
6,http://www.wikidata.org/entity/Q16069681,Q16069681,None
7,http://www.wikidata.org/entity/Q7773994,The Weapon Too Dreadful to Use,1939-01-01T00:00:00Z
8,http://www.wikidata.org/entity/Q7838296,Trends,1939-01-01T00:00:00Z
9,http://www.wikidata.org/entity/Q5641557,Half-Breed,1940-01-01T00:00:00Z


### Consulta 8

Asimov es sin duda un autor prolífico. ¿Cuántas obras escribió a lo largo de su vida? Vamos a considerar sólo instancias directas de [literary work (Q7725634)](https://www.wikidata.org/wiki/Q7725634).

In [11]:
%%wdsparql
SELECT (COUNT (*) as ?totalcount)
WHERE
{   
    # ?obra  instance of   literary work
    ?obra    wdt:P31       wd:Q7725634;
    #        autor      Isaac Asimov
             wdt:P50       wd:Q34981.
}

,totalcount
0,272


### Consulta 9

De todas las obras que escribió a lo largo de su vida, queremos recuperar los nombres y fechas de publicación de aquellas escritas entre 1970 y 1980. Si alguna obra tiene varias fechas de publicación, se mostrará si alguna de ellas está dentro de ese periodo.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#FILTER) te resulte útil.

In [12]:
%%wdsparql
SELECT ?obra ?titulo ?fecha 
WHERE
{   
   # ?obra  instance of   literary work
   ?obra    wdt:P31       wd:Q7725634;
   #        autor         Isaac Asimov
            wdt:P50       wd:Q34981;
   #        titulo        ?titulo
            wdt:P1476     ?titulo;
   #        duration      ?fecha
            wdt:P577      ?fecha.
  FILTER(YEAR(?fecha) >= 1970 && YEAR(?fecha) <= 1980)
}
ORDER BY ASC(?fecha)

,obra,titulo,fecha
0,http://www.wikidata.org/entity/Q4631777,2430 n. Chr.,1970-01-01T00:00:00Z
1,http://www.wikidata.org/entity/Q4631777,2430 A. D.,1970-01-01T00:00:00Z
2,http://www.wikidata.org/entity/Q4806996,Asimov's Guide to Shakespeare,1970-01-01T00:00:00Z
3,http://www.wikidata.org/entity/Q7765145,The Solar System and Back,1970-01-19T00:00:00Z
4,http://www.wikidata.org/entity/Q28808642,A Problem of Numbers,1970-05-01T00:00:00Z
5,http://www.wikidata.org/entity/Q28808642,As Chemist to Chemist,1970-05-01T00:00:00Z
6,http://www.wikidata.org/entity/Q16932955,The Stars in their Courses,1971-01-01T00:00:00Z
7,http://www.wikidata.org/entity/Q389750,The Gods Themselves,1972-01-01T00:00:00Z
8,http://www.wikidata.org/entity/Q831556,Mirror Image,1972-01-01T00:00:00Z
9,http://www.wikidata.org/entity/Q6404541,Kid Stuff,1972-01-01T00:00:00Z


### Consulta 10

¿Qué otros escritores de ciencia ficción estudiaron en algún centro donde también estudió Asimov? Para cada uno de ellos muestra su nombre y fechas de nacimiento y defunción (si están disponibles) y el centro donde estudió. Muestra los resultados ordenados alfabéticamente por centro y luego por autor.

### Consulta 11

¿Cuántos asertos hay sobre Asimov en Wikidata? Ten en cuenta que Asimov puede aparece tanto como sujeto como objeto de cada tripleta.

In [14]:
%%wdsparql
SELECT (COUNT(*) AS ?nasserts)
WHERE
{   
    #Hacemos el bind de asimov a una variable (?asimov)
    BIND(wd:Q34981 AS ?asimov)
    #Asertos donde Asimov es el sujeto
    ?asimov ?assert1 ?item.
    #Asertos donde Asimov es el objeto
    ?subject    ?assert2 ?asimov.
}

,nasserts
0,1221380


__Fecha de las consultas: 21 de abril de 2022__